In [1]:
import pandas as pd
import cv2
import opencv_jupyter_ui as jcv2
from train_model import train_model
from feat import Detector
import warnings
import time
warnings.filterwarnings(action="ignore", category=FutureWarning)


In [2]:
model = train_model()
detector = Detector(device="cuda")

Train size: 903 Val size: 258 Test size: 129
Validation accuracy: 59.69 %, with 17 neighbors
Test accuracy: 62.79 %, with 17 neighbors


In [4]:
cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_BUFFERSIZE, 1)

last_detection_time = time.time()
emotion = None

while True:
    ret, frame = cam.read()
    if not ret:
        print("OpenCV found an error reading the next frame.")
        break

    current_time = time.time()  # Use a different variable name
    
    if current_time - last_detection_time >= 2:
        faces = detector.detect_faces(frame)
        landmarks = detector.detect_landmarks(frame, faces)    
        action_units = detector.detect_aus(frame, landmarks)
        action_units = action_units[0]
    
        # Predict emotion
        emotion = model.predict(action_units)
        last_detection_time = current_time
    if emotion is not None:
        cv2.putText(frame, emotion[0], (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    jcv2.imshow("Emotion Detection", frame)

    key = jcv2.waitKey(1) & 0xFF
    if key == 27: # ESC pressed
        break

cam.release()
jcv2.destroyAllWindows()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
#cam.release()